# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [5]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


nltk.download(['punkt', 'wordnet'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
pd.options.display.max_columns = None

In [7]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("disaster_messages", con=engine)

In [8]:
X = df['message']
Y = df.iloc[:, 4:]

In [9]:
print(df.shape)
print(Y.shape)

(26216, 40)
(26216, 36)


In [10]:
df[:1]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,water-0,food-0,shelter-0,clothing-0,money-0,missing_people-0,refugees-0,death-0,other_aid-0,infrastructure_related-0,transport-0,buildings-0,electricity-0,tools-0,hospitals-0,shops-0,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [11]:
Y[:1]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,water-0,food-0,shelter-0,clothing-0,money-0,missing_people-0,refugees-0,death-0,other_aid-0,infrastructure_related-0,transport-0,buildings-0,electricity-0,tools-0,hospitals-0,shops-0,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 2. Write a tokenization function to process your text data

In [12]:
def tokenize(text):
    
    
    """ tokenize text"""
  
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens=[]
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [15]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
y_pred = pipeline.predict(X_test)

In [17]:
def test_model(y_test, y_pred):
    
    """
    Function to iterate through columns and call sklearn classification report on each.
    """
    for index, column in enumerate(y_test):
        print(column, classification_report(y_test[column], y_pred[:, index]))

In [18]:
test_model(y_test, y_pred)

related              precision    recall  f1-score   support

  related-0       0.65      0.38      0.48      1516
  related-1       0.83      0.94      0.88      4988
  related-2       0.87      0.26      0.40        50

avg / total       0.79      0.80      0.78      6554

request              precision    recall  f1-score   support

  request-0       0.88      0.98      0.93      5429
  request-1       0.81      0.38      0.52      1125

avg / total       0.87      0.88      0.86      6554

offer              precision    recall  f1-score   support

    offer-0       1.00      1.00      1.00      6529
    offer-1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6554

aid_related                precision    recall  f1-score   support

aid_related-0       0.71      0.87      0.78      3798
aid_related-1       0.74      0.52      0.61      2756

  avg / total       0.73      0.72      0.71      6554



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


medical_help                 precision    recall  f1-score   support

medical_help-0       0.92      0.99      0.96      6012
medical_help-1       0.53      0.07      0.12       542

   avg / total       0.89      0.92      0.89      6554

medical_products                     precision    recall  f1-score   support

medical_products-0       0.95      1.00      0.98      6215
medical_products-1       0.73      0.11      0.19       339

       avg / total       0.94      0.95      0.93      6554

search_and_rescue                      precision    recall  f1-score   support

search_and_rescue-0       0.97      1.00      0.98      6350
search_and_rescue-1       0.70      0.07      0.13       204

        avg / total       0.96      0.97      0.96      6554

security              precision    recall  f1-score   support

 security-0       0.98      1.00      0.99      6422
 security-1       0.20      0.01      0.01       132

avg / total       0.96      0.98      0.97      6554

military   

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f119d575158>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [20]:
# specify parameters for grid search 
parameters = {
    'clf__estimator__n_estimators' : [50, 100]
}

In [21]:
# create grid search object 
cv = GridSearchCV(pipeline, param_grid=parameters)

cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [22]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [23]:
cv.best_params_

{'clf__estimator__n_estimators': 100}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
y_pred = cv.predict(X_test)

In [25]:
test_model(y_test, y_pred)

related              precision    recall  f1-score   support

  related-0       0.75      0.28      0.41      1516
  related-1       0.81      0.97      0.88      4988
  related-2       0.71      0.10      0.18        50

avg / total       0.79      0.80      0.77      6554

request              precision    recall  f1-score   support

  request-0       0.89      0.99      0.94      5429
  request-1       0.88      0.42      0.57      1125

avg / total       0.89      0.89      0.87      6554

offer              precision    recall  f1-score   support

    offer-0       1.00      1.00      1.00      6529
    offer-1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6554

aid_related                precision    recall  f1-score   support

aid_related-0       0.75      0.87      0.81      3798
aid_related-1       0.77      0.61      0.68      2756

  avg / total       0.76      0.76      0.76      6554



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


medical_help                 precision    recall  f1-score   support

medical_help-0       0.92      1.00      0.96      6012
medical_help-1       0.56      0.03      0.06       542

   avg / total       0.89      0.92      0.88      6554

medical_products                     precision    recall  f1-score   support

medical_products-0       0.95      1.00      0.97      6215
medical_products-1       0.76      0.06      0.10       339

       avg / total       0.94      0.95      0.93      6554

search_and_rescue                      precision    recall  f1-score   support

search_and_rescue-0       0.97      1.00      0.98      6350
search_and_rescue-1       0.69      0.04      0.08       204

        avg / total       0.96      0.97      0.96      6554

security              precision    recall  f1-score   support

 security-0       0.98      1.00      0.99      6422
 security-1       0.33      0.01      0.01       132

avg / total       0.97      0.98      0.97      6554

military   

In [26]:
accuracy = (y_pred == y_test).mean()
accuracy

related                   0.804699
request                   0.890601
offer                     0.996186
aid_related               0.761062
medical_help              0.917913
medical_products          0.950259
search_and_rescue         0.969637
security                  0.979707
military                  0.966433
child_alone               1.000000
water                     0.951175
food                      0.929204
shelter                   0.927830
clothing                  0.984132
money                     0.974367
missing_people            0.986726
refugees                  0.965517
death                     0.958804
other_aid                 0.862527
infrastructure_related    0.929356
transport                 0.953311
buildings                 0.951022
electricity               0.981233
tools                     0.993287
hospitals                 0.985810
shops                     0.994507
aid_centers               0.988404
other_infrastructure      0.952090
weather_related     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [27]:
import pickle
filename = 'model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.